# Split voxel features
In order to speed up computing we computed the voxel features by batch

We now need to split those and add the infos the the dataset

We already have relaxed_wild_3D_path and relaxed_mutated_3D_path and now want voxel_direct_path and voxel_reversed_path as well


In [1]:
import numpy as np
import pandas as pd
from glob import glob
import re


In [2]:
DATASET_INPUT = "../data/main_dataset_creation/outputs/all_v3/dataset_with_all_features.csv"
SUBMISSION_INPUT = "../data/main_dataset_creation/outputs/all_v3/submission_all_features_filled_nan.csv"

DATASET_OUTPUT = "../data/main_dataset_creation/outputs/all_v3/dataset_with_voxel.csv"
SUBMISSION_OUTPUT = "../data/main_dataset_creation/outputs/all_v3/submission_with_voxel.csv"
SUBMISSION_OUTPUT_WITH_KAGGLE = "../data/main_dataset_creation/outputs/all_v3/submission_with_voxel_from_kaggle.csv"

SPLIT_KAGGLE_FEATURES = True

In [3]:
train_df = pd.read_csv(DATASET_INPUT)
submission_df = pd.read_csv(SUBMISSION_INPUT)


In [15]:
if SPLIT_KAGGLE_FEATURES:
    TEST_FEATURES_PATH = "../data/main_dataset_creation/3D_structures/AF70_kaggle_features.npy"
    output_dir = "./splitted_voxel_features/"
    uniprot = "kaggle_AF70"

    submission_df = pd.read_csv(SUBMISSION_OUTPUT)
    submission_df["kaggle_voxel_path"] = ""
    print(len(submission_df))


    test_features = np.load(TEST_FEATURES_PATH)
    test_features = np.moveaxis(test_features, 1, -1)
    print(test_features.shape)

    def operation_type(row):
        if row["mutated_aa"] == "-":
            return "deletion"
        elif row["mutated_aa"] == row["wild_aa"]:
            return "none"
        else:
            return "mutation"

    submission_df["operation"] = submission_df.apply(operation_type, axis=1)

    kaggle_voxel_paths = []
    for _, row in submission_df.loc[submission_df.operation == "mutation"].iterrows():
        mutation_code = row["wild_aa"]+str(row["mutation_position"])+row["mutated_aa"]
        direct_path = output_dir+f"{uniprot}_{mutation_code}_direct.npy"
        kaggle_voxel_paths.append(direct_path)

    print(len(kaggle_voxel_paths))
    for i, direct_path in enumerate(kaggle_voxel_paths):
        np.save(direct_path, test_features[i])
    submission_df.loc[submission_df.operation ==
                      "mutation", "kaggle_voxel_path"] = kaggle_voxel_paths
    submission_df.loc[submission_df.operation ==
                      "deletion", "kaggle_voxel_path"] = submission_df.loc[submission_df.operation ==
                                                                           "deletion", "direct_voxel_path"]
    print(submission_df["kaggle_voxel_path"].head())
    submission_df.to_csv(SUBMISSION_OUTPUT_WITH_KAGGLE, index=False)
 

2413
(2335, 16, 16, 16, 14)
2335
0    ./splitted_voxel_features/kaggle_AF70_L38E_dir...
1    ./splitted_voxel_features/kaggle_AF70_L38K_dir...
2       ./splitted_voxel_features/deletion_L17__direct
3    ./splitted_voxel_features/kaggle_AF70_K39C_dir...
4    ./splitted_voxel_features/kaggle_AF70_K39F_dir...
Name: kaggle_voxel_path, dtype: object


In [4]:
def split_features(name: str, base_dir="./", output_dir="./splitted_voxel_features/"):
    """
    function that creates a df identification infos and the avg prediction values of thermonet
    (both direct and reversed)

    returns:
    a pandas.DataFrame which has the columns:
    ['wild_path', 'position', 'mutated_path', 'direct_thermonet', 'reversed_thermonet']
    """
    uniprot = name.split("_")[0]
    variant_path = base_dir+"gends_input/"+name+"_variants.txt"
    base_features_path = base_dir+"gends_output/"+name

    # first we get the variant list in a df, in order to know to which mutations each value corresponds to
    variant_df = pd.read_csv(variant_path,
                             names=["wild_path", "position", "mutated_path"],
                             sep=' ')
    variant_df.position = variant_df.position.apply(lambda x: x-1).astype(int)
    variant_df = pd.concat([variant_df, pd.DataFrame(columns=["direct_voxel_path",
                                                              "reversed_voxel_path"])],
                           axis=1)

    # then we load the features from the gends outputs
    try:
        direct_features = np.load(
            base_features_path+"_stacked_16_1_direct.npy")
        reversed_features = np.load(
            base_features_path+"_stacked_16_1_reversed.npy")
    except Exception as e:
        print(f"Exception raised for {name}, {base_features_path}: {e}")
        print(f"not adding features for {name}")
        return pd.DataFrame()

    if len(variant_df) != direct_features.shape[0]:
        print(
            f"ERROR for {name}, {base_features_path}, len(variant_df) != direct_features.shape[0]")
    for i, row in variant_df.iterrows():
        # ./compute_mutated_structures/relaxed_pdb/P03050_relaxed/P03050_relaxed_P8A_relaxed.pdb
        # => P8A
        mutated_path = row["mutated_path"]
        try:
            result = re.search('_relaxed_(.*)_relaxed', mutated_path)
            mutation_code = result.group(1)
        except:
            # case where we have: compute_mutated_structures/relaxed_pdb/AF70_alphafold/K212__unrelaxed_rank_1_model_3_relaxed.pdb
            result = re.search('_alphafold/(.*)_unrelaxed', mutated_path)
            mutation_code = result.group(1)

        direct = direct_features[i]
        reversed = reversed_features[i]
        direct_path = output_dir+f"{uniprot}_{mutation_code}_direct"
        reversed_path = output_dir+f"{uniprot}_{mutation_code}_reversed"

        variant_df.loc[i, "direct_voxel_path"] = direct_path
        variant_df.loc[i, "reversed_voxel_path"] = reversed_path
        np.save(direct_path, direct)
        np.save(reversed_path, reversed)

    return variant_df


def update_main_df(row, main_df: pd.DataFrame):
    # we get mutated_path as a unique protein+mutation identifier
    # but multiple record could have the same mutation on the same protein
    # for example same mutation at different pH

    mutated_path = row["mutated_path"]
    main_df.loc[
        (main_df.relaxed_mutated_3D_path.eq(mutated_path)),
        "direct_voxel_path"
    ] = row["direct_voxel_path"]
    main_df.loc[
        (main_df.relaxed_mutated_3D_path.eq(mutated_path)),
        "reversed_voxel_path"
    ] = row["reversed_voxel_path"]

    return row


def split_voxel_features(main_df: pd.DataFrame, base_dir="./"):
    main_df = pd.concat([main_df, pd.DataFrame(columns=["direct_voxel_path",
                                                        "reversed_voxel_path"])],
                        axis=1)

    all_variants = glob(
        f"{base_dir}gends_input/*_variants.txt")
    all_names = [v.split('/')[-1].split("_variants.txt")[0]
                 for v in all_variants]
    for name in all_names:
        variant_df = split_features(
            name, base_dir=base_dir)
        variant_df.apply(lambda row: update_main_df(row, main_df), axis=1)

    return main_df


In [5]:
train_df = split_voxel_features(train_df)
submission_df = split_voxel_features(submission_df)
train_df.to_csv(DATASET_OUTPUT, index=False)
submission_df.to_csv(SUBMISSION_OUTPUT, index=False)


Exception raised for O60885_260, ./gends_output/O60885_260: [Errno 2] No such file or directory: './gends_output/O60885_260_stacked_16_1_direct.npy'
not adding features for O60885_260
Exception raised for P02751_70, ./gends_output/P02751_70: [Errno 2] No such file or directory: './gends_output/P02751_70_stacked_16_1_direct.npy'
not adding features for P02751_70
Exception raised for O60885_260, ./gends_output/O60885_260: [Errno 2] No such file or directory: './gends_output/O60885_260_stacked_16_1_direct.npy'
not adding features for O60885_260
Exception raised for P02751_70, ./gends_output/P02751_70: [Errno 2] No such file or directory: './gends_output/P02751_70_stacked_16_1_direct.npy'
not adding features for P02751_70


In [6]:
train_df = pd.read_csv(DATASET_OUTPUT)
submission_df = pd.read_csv(SUBMISSION_OUTPUT)


In [7]:
train_df = train_df[~(train_df.direct_voxel_path.isna())]
train_df.to_csv(DATASET_OUTPUT, index=False)


In [8]:
train_df = pd.read_csv(DATASET_OUTPUT)
train_df.direct_voxel_path.isna().sum()

0

In [9]:
import re
s = "compute_mutated_structures/relaxed_pdb/AF70_alphafold/K212__unrelaxed_rank_1_model_3_relaxed.pdb"
result = re.search('_alphafold/(.*)_unrelaxed', s)
result.group(1)

'K212_'